##### Začneme tím, že na-import-ujeme knihovny:

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

##### Začneme stahování od roku 2018 a postupně budeme přidávat starší soubory.

In [2]:
response = requests.get('http://www.sukl.cz/2012')

In [3]:
soup = BeautifulSoup(response.text, "lxml")

Odkazy v HTML se začínají písmenkem a mají atribut "href".  Zároveň při prvním pohledu na stránku http://www.sukl.cz/2018 vidíme, že názvy složek, které potřebujeme se začínají slovem "Distribuce":

In [4]:
links = soup.find_all('a')

In [5]:
for slovo in links:
    
    if slovo.text.startswith('DistribuceLP_'):
        x ='http://www.sukl.cz' + slovo.attrs['href'] #Po několika pokusů jsme zjistily, že potřebujeme přidat 'http://www.sukl.cz'
        print(x) #Vytiskneme každý odkaz, abychom mohly prověřit, jestli opravdu fungují

http://www.sukl.cz/file/74315_1_1
http://www.sukl.cz/file/74314_1_1
http://www.sukl.cz/file/74313_1_1
http://www.sukl.cz/file/74312_1_1
http://www.sukl.cz/file/74311_1_1
http://www.sukl.cz/file/74310_1_1
http://www.sukl.cz/file/74309_1_1
http://www.sukl.cz/file/74308_1_1
http://www.sukl.cz/file/74307_1_1
http://www.sukl.cz/file/74306_1_1
http://www.sukl.cz/file/74305_1_1
http://www.sukl.cz/file/76216_1_1


##### Vytvoříme seznam a nahrajeme do toho naší odkazy:

In [6]:
our_list = ['']
i = 0
for slovo in links:
    if slovo.text.startswith('DistribuceLP_'):
        x ='http://www.sukl.cz' + slovo.attrs['href']
        if i == 0:
            our_list[i] = x
        else:
            our_list.append(x)
        i+=1
print(our_list)

['http://www.sukl.cz/file/74315_1_1', 'http://www.sukl.cz/file/74314_1_1', 'http://www.sukl.cz/file/74313_1_1', 'http://www.sukl.cz/file/74312_1_1', 'http://www.sukl.cz/file/74311_1_1', 'http://www.sukl.cz/file/74310_1_1', 'http://www.sukl.cz/file/74309_1_1', 'http://www.sukl.cz/file/74308_1_1', 'http://www.sukl.cz/file/74307_1_1', 'http://www.sukl.cz/file/74306_1_1', 'http://www.sukl.cz/file/74305_1_1', 'http://www.sukl.cz/file/76216_1_1']


##### Vytvoříme seznam "our_files", do kterého nahrajeme každý ze souboru s pomocí seznamu odkazů:

In [7]:
our_files = [ pd.read_excel(x, encoding='utf-8') for x in our_list ]

In [8]:
our_filesX = our_files

##### Abychom mohly prověřit, jestli všechno je správně, vypišeme počet řádků v každém že souborů a pak součet:

In [9]:
number_of_rows =[0]*12 #budeme používat seznam obsahující 12 čísel (protože 12 měsíců)

for i in range(0, len(our_files)):
    number_of_rows[i] = our_files[i].shape[0]
number_of_rows

[22656,
 22815,
 23021,
 22629,
 23175,
 23136,
 23041,
 23248,
 23115,
 24088,
 24212,
 23370]

In [10]:
sum(number_of_rows) 

278506

### Vsimly jsme si, ze tabulky pro rok 2011 a 2012 nemaji prvni sloupec "Obdobi" a proto pridame ten sloupec ke kazde z tabulek. Abychom zachovaly stejné netradiční formát datumu, vyplnily ten sloupec jako “201x.MM”, kde x značí rok 1 (2011) nebo 2 (2012) a MM značí měsíc. 

Abychom zachovaly stejné netradiční formát datumu, přidaly jsme do tabulek sloupec a vyplnily ho jako “201x.MM”, kde x značí rok 2011 nebo 2012 a MM značí měsíc. Pro vyplnění období uvnitř cyklu jsme použily metodu “.format”, která automaticky doplňovala číslo měsíců. (Pozn.: Chtěly jsme prvně spojit tabulky a následně až řešit problém špatného formátu.)

In [11]:
for i in range(0, len(our_files)):
    our_files[i].insert(loc=0, column='Období', value='2012.{:02d}'.format(i+1))

##### Teď konečně spojíme složky. Víme, že mají stejnou strukturu (počet a názvy sloupců), proto nahrajeme soubory jeden za druhým. Budeme postupovat podle vzoru:

year_201x = pd.concat([our_files[0], our_files[1], "a tak dale" ], ignore_index = True), kde x je 2012 anebo 2012

In [12]:
for i in range(1, len(our_files)):
    our_files[i] = pd.concat([our_files[i-1], our_files[i]], ignore_index = True)
year_2012 = our_files[i]
year_2012.head()

,Období,Typ odběratele,ATC,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet denních definovaných dávek/ balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
0,2012.01,LEKARNA,A01A,53907,SEPTOMIXINE,STM PST 1X7.5GM,SPH,F,D,49,897.26,43965.74,NaN,NaN,R
1,2012.01,DISTRIBUTOR_CR,A01AA01,211,NATRIUM FLUORATUM SLOVAKOFARMA,POR TBL NOB 250X0.55MG,ZEH,SK,D,2839,NaN,NaN,125.0,354875.0,F
2,2012.01,LEKARNA,A01AA01,211,NATRIUM FLUORATUM SLOVAKOFARMA,POR TBL NOB 250X0.55MG,ZEH,SK,D,2853,60.79,173433.87,125.0,356625.0,F
3,2012.01,LEKARNA,A01AA01,211,NATRIUM FLUORATUM SLOVAKOFARMA,POR TBL NOB 250X0.55MG,ZEH,SK,V,-28,60.79,-1702.12,125.0,-3500.0,F
4,2012.01,VETERINARNI_LEKAR,A01AA01,211,NATRIUM FLUORATUM SLOVAKOFARMA,POR TBL NOB 250X0.55MG,ZEH,SK,D,4,60.79,243.16,125.0,500.0,F


Zkontrolujeme, jestli počet řádků je stejný, jak v "sum(number_of_rows)", což je 72154 pro rok 2018

In [13]:
year_2012.shape

(278506, 15)

In [14]:
year_2012.tail()

,Období,Typ odběratele,ATC,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet denních definovaných dávek/ balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
278501,2012.12,LEKARNA,V12,70693,VISCUM ALBUM,POR GRA 4GM 2CH-200CH,LBN,F,D,10,54.60,546.00,NaN,NaN,R
278502,2012.12,LEKARNA,V12,82617,VISCUM ALBUM,POR GRA 4GM 3K-10MK,LBN,F,D,1,52.44,52.44,NaN,NaN,R
278503,2012.12,DISTRIBUTOR_CR,V12,72175,ZINCUM METALLICUM,POR GRA 4GM CH3-CH200,LBN,F,D,279,NaN,NaN,NaN,NaN,R
278504,2012.12,LEKARNA,V12,72175,ZINCUM METALLICUM,POR GRA 4GM CH3-CH200,LBN,F,D,257,54.64,14042.48,NaN,NaN,R
278505,2012.12,LEKARNA,V12,72175,ZINCUM METALLICUM,POR GRA 4GM CH3-CH200,LBN,F,V,-5,54.64,-273.20,NaN,NaN,R


##### Hura! Ted muzeme exportovat tuto slozku:

In [15]:
year_2012.to_csv('years/year_2012.csv', index = False, encoding='utf-8')

###### Pro rok 2011 pouzijeme stejny postup!